In [1]:
import requests
import json
import pandas as pd

In [11]:
def fetch_game_ids(years) -> pd.DataFrame:
    df = pd.DataFrame()
    for year in years:
        url = f'https://stats.nba.com/stats/leaguegamelog?Counter=1000&DateFrom=&DateTo=&Direction=DESC&ISTRound=&LeagueID=00&PlayerOrTeam=T&Season=20{year-1}-{year}&SeasonType=Regular%20Season&Sorter=DATE'
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64)', 'x-nba-stats-origin': 'stats', 'x-nba-stats-token': 'true', 'Host':'stats.nba.com', 'Referer':f'https://www.nba.com/'}
        r= requests.get(url, headers=headers, timeout = 7)
        data = json.loads(r.text)
        games = data['resultSets'][0]['rowSet']
        ids = [[info[4],info[5],info[6].split(' @ ')[0],info[6].split(' @ ')[1]] for info in games if '@' in info[6]]
        ids = sorted(ids)
        temp_df = pd.DataFrame.from_dict(ids)
        temp_df.columns=[['game_id','date','home','away']]
        temp_df['season'] = year
        if df.empty:
            df = temp_df.copy(deep=True)
        else:
            df = pd.concat((df,temp_df))
    df
    df['game_id'] = df['game_id'].astype(str)
    df.to_csv('game_ids.csv',index=False)
    return df
fetch_game_ids(range(20,25))

In [26]:
for x in range(23,25):
    print(x)

23
24


In [13]:
game_ids = pd.read_csv('game_ids.csv',dtype={'game_id':'str'})

In [23]:
import os
import datetime as dt
update_date = dt.date.fromtimestamp(os.path.getmtime('game_ids.csv'))

In [25]:
update_date == dt.date.today()

True